In [27]:
from dotenv import load_dotenv
from anthropic import Anthropic
import ast
import json
import re
from statistics import mean

load_dotenv()
client = Anthropic()
model = "claude-sonnet-4-0"

In [21]:
def add_user_message(messages, text):
    messages.append({"role": "user", "content": text})
    return messages

def add_assistant_message(messages, text):
    messages.append({"role": "assistant", "content": text})
    return messages

def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
            "model": model,
            "max_tokens": 1000,
            "messages": messages,
            "temperature":temperature,
            "stop_sequences":stop_sequences,
    }
    if system:
        params["system"] = system

    message = client.messages.create(
        **params
    )
    return message.content[0].text


In [22]:
def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
        "format": "json" or "python" or "regex"
        "solution_criteria": "Description of criteria as a list from which to score if this is a good solution"
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```json")
    text = chat(messages, stop_sequences=["```"])
    return json.loads(text)

In [23]:
dataset = generate_dataset()
with open("data/dataset.json", "w") as f:
    json.dump(dataset, f, indent=2)
dataset

[{'task': 'Create an IAM policy that allows read-only access to all S3 buckets and objects in an AWS account',
  'format': 'json',
  'solution_criteria': ['Uses correct IAM policy structure with Version, Statement fields',
   'Includes proper S3 read actions (s3:GetObject, s3:ListBucket, etc.)',
   'Uses appropriate resource ARN format for S3 (arn:aws:s3:::* and arn:aws:s3:::*/*)',
   "Effect is set to 'Allow'",
   'Does not include write or delete permissions']},
 {'task': 'Write a Python function that takes an AWS S3 object key and returns True if it represents a log file based on common log file patterns',
  'format': 'python',
  'solution_criteria': ['Function accepts a single string parameter (S3 object key)',
   'Returns boolean value (True/False)',
   'Checks for common log file extensions (.log, .txt, .out)',
   'Handles case-insensitive matching',
   "Considers common log file naming patterns (contains 'log', 'access', 'error', etc.)",
   'Function is syntactically correct Pyt

In [24]:
def grade_by_model(test_case, output):
    # Create evaluation prompt
    eval_prompt = f'''
    You are an expert code reviewer. Evaluate this AI-generated solution.
    
    Original Task:
    <task>
    {test_case["task"]}
    </task>
    
    Solution to Evaluate:
    <solution>
    {output}
    </solution>

    Solution Criteria for Scoring:
    <solution_criteria>
    {test_case["solution_criteria"]}
    </solution_criteria>
    
    Provide your evaluation as a structured JSON object with:
    - "strengths": An array of 1-3 key strengths
    - "weaknesses": An array of 1-3 key areas for improvement  
    - "reasoning": A concise explanation of your assessment
    - "score": A number between 1-10
    Respond with JSON. Keep your response concise and direct.
    Example response shape:
    {{
        "strengths": string[],
        "weaknesses": string[],
        "reasoning": string,
        "score": number
    }}
    '''
    
    messages = []
    add_user_message(messages, eval_prompt)
    add_assistant_message(messages, "```json")
    
    eval_text = chat(messages, stop_sequences=["```"])
    return json.loads(eval_text)

def validate_json(text):
    try:
        json.loads(text.strip())
        return 10
    except json.JSONDecodeError:
        return 0


def validate_python(text):
    try:
        ast.parse(text.strip())
        return 10
    except SyntaxError:
        return 0


def validate_regex(text):
    try:
        re.compile(text.strip())
        return 10
    except re.error:
        return 0


def grade_syntax(response, test_case):
    format = test_case["format"]
    if format == "json":
        return validate_json(response)
    elif format == "python":
        return validate_python(response)
    else:
        return validate_regex(response)


In [25]:
def run_prompt(test_case):
    """Merges the prompt and test case input, then returns the result"""
    
    prompt = f"""Please solve the following task:

    {test_case["task"]}
    * Respond only with Python, JSON, or a plain Regex
    * Do not add any comments or commentary or explanation
    """

    messages=[]
    add_user_message(messages, prompt)
    add_assistant_message(messages, "```code")
    output = chat(messages, stop_sequences=["```"])
    return output
    

def run_test_case(test_case):
    """Calls run_prompt, then grades the result"""
    output = run_prompt(test_case)

    # GRADE
    model_grade = grade_by_model(test_case, output)
    model_score = model_grade["score"]
    reasoning = model_grade["reasoning"]

    syntax_score = grade_syntax(output, test_case)
    score = (model_score + syntax_score) /2
    return {
    "output": output,
    "test_case": test_case,
    "score": score,
    "reasoning":reasoning
    }
    
def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""
    results = []

    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)

    average_score = mean([result["score"] for result in results])
    print(f"Average score: {average_score}")

    return results

In [28]:
results = run_eval(dataset)

Average score: 8.5


In [29]:
print(json.dumps(results, indent=2))

[
  {
    "output": "\n{\n  \"Version\": \"2012-10-17\",\n  \"Statement\": [\n    {\n      \"Effect\": \"Allow\",\n      \"Action\": [\n        \"s3:GetObject\",\n        \"s3:GetObjectVersion\",\n        \"s3:ListBucket\",\n        \"s3:ListBucketVersions\",\n        \"s3:GetBucketLocation\",\n        \"s3:GetBucketVersioning\",\n        \"s3:ListAllMyBuckets\"\n      ],\n      \"Resource\": [\n        \"arn:aws:s3:::*\",\n        \"arn:aws:s3:::*/*\"\n      ]\n    }\n  ]\n}\n",
    "test_case": {
      "task": "Create an IAM policy that allows read-only access to all S3 buckets and objects in an AWS account",
      "format": "json",
      "solution_criteria": [
        "Uses correct IAM policy structure with Version, Statement fields",
        "Includes proper S3 read actions (s3:GetObject, s3:ListBucket, etc.)",
        "Uses appropriate resource ARN format for S3 (arn:aws:s3:::* and arn:aws:s3:::*/*)",
        "Effect is set to 'Allow'",
        "Does not include write or delete pe